In [9]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
from torchvision import models, datasets
from torch.utils.data import random_split, DataLoader

import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats("svg")
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (10, 5)

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import numpy as np

In [10]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
dataset = datasets.ImageFolder('data/Dartmouth_Clean', transform=transform)

total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

In [14]:
# model = models.resnet18(pretrained=True)
model = models.efficientnet_b0(pretrained=True)

num_classes = 8

# model.fc = nn.Linear(model.fc.in_features, num_classes)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, num_classes)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [15]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [16]:
num_epochs = 15
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []

for epoch in range(num_epochs):
    
    model.train()
    total_loss, total_correct = 0, 0

    
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        output = model(images)
        loss = criterion(output, labels)
        
        optimizer.zero_grad()  
        loss.backward()        
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total_correct += (predicted == labels).sum().item()
        
    print("hello")
        
    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    
    avg_train_accuracy = total_correct / len(train_loader.dataset)
    train_accuracies.append(avg_train_accuracy)
    
    # scheduler.step()
    
    model.eval()
    total_val_loss, total_val_correct = 0, 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            loss = criterion(output, labels)
            total_val_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total_val_correct += (predicted == labels).sum().item()
            
    avg_val_loss = total_val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    
    val_accuracy = total_val_correct / len(val_loader.dataset)
    val_accuracies.append(val_accuracy)
    
    print(f'''
    Epoch: {epoch+1}/{num_epochs}
    Training   - Loss: {avg_train_loss:.4f}, Accuracy: {avg_train_accuracy:.4f}
    Validation - Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.4f}
    ''')

hello

    Epoch: 1/15
    Training   - Loss: 1.7550, Accuracy: 0.3885
    Validation - Loss: 1.0886, Accuracy: 0.6447
    


KeyboardInterrupt: 

In [ ]:


# Plotting training and validation accuracy
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(range(num_epochs), train_accuracies, label='Training Accuracy')
plt.plot(range(num_epochs), val_accuracies, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plotting training and validation loss
plt.subplot(1, 2, 2)
plt.plot(range(num_epochs), train_losses, label='Training Loss')
plt.plot(range(num_epochs), val_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:


# Initialize lists to store true labels and predictions
all_labels = []
all_predictions = []

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        d
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Compute the confusion matrix
cm = confusion_matrix(all_labels, all_predictions)
# Normalize the confusion matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


In [ ]:
emotion_labels = ["Afraid", "Angry", "Disgusted", "Happy", "Neutral", "Please", "Sad", "Surprised"]
plt.figure(figsize=(10, 8))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=emotion_labels, yticklabels=emotion_labels)
plt.title('Normalized Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()


In [ ]:
class_report = classification_report(all_labels, all_predictions, target_names=emotion_labels)
print(class_report)
